In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
filepath = "translation_comparison_20250807-0946.csv"
df = pd.read_csv(filepath)

df.rename(columns={"cosine_similarity_vs_target": "similarity_to_old_translation"}, inplace=True)
df['quality_vs_tb'] = df['cosine_similarity_vs_source'] - df['cosine_similarity_original_translation']

In [3]:
def make_quantile(q):
    return lambda x: x.quantile(q)

percentiles = [0.01, 0.1, 0.5, 0.9, 0.99]
agg_funcs = ['mean', 'min', 'max'] + [make_quantile(q) for q in percentiles]
agg_names = ['mean', 'min', 'max'] + [f"{int(q*100)}%" for q in percentiles]

print("\nQuality of Translations Versus Translation Bureau")
display(df.groupby('translator_name')['quality_vs_tb'].agg(agg_funcs).reset_index().set_axis(
    ['translator_name'] + agg_names, axis=1
).T)

print("\nSimilarity to Translation Bureau Translation")
display(df.groupby('translator_name')['similarity_to_old_translation'].agg(agg_funcs).reset_index().set_axis(
    ['translator_name'] + agg_names, axis=1
).T)


Quality of Translations Versus Translation Bureau


,0,1,2,3
translator_name,m2m100_418m,mbart50_mmt,nllb_3b_researchonly,opus_mt
mean,0.03,0.03,0.03,0.03
min,-0.74,-0.23,-0.85,-0.16
max,0.40,0.41,0.41,0.41
1%,-0.08,-0.05,-0.08,-0.05
10%,-0.02,-0.01,-0.01,-0.01
50%,0.02,0.02,0.02,0.02
90%,0.09,0.08,0.08,0.08
99%,0.17,0.17,0.17,0.16



Similarity to Translation Bureau Translation


,0,1,2,3
translator_name,m2m100_418m,mbart50_mmt,nllb_3b_researchonly,opus_mt
mean,0.92,0.93,0.93,0.94
min,0.16,0.47,0.02,0.55
max,1.00,1.00,1.00,1.00
1%,0.73,0.76,0.75,0.77
10%,0.85,0.87,0.87,0.88
50%,0.93,0.94,0.95,0.95
90%,0.98,0.99,0.99,0.99
99%,1.00,1.00,1.00,1.00


In [ ]:
# opus is the best, but all are good

# how do i choose a model if they all perform well and they're not very different?
OPTIONS:
* just do opus
* finetune all of them, and recheck performance
* pick the smallest one
* best bad values - highest min / 1%
* best total values (even if they're very close)